In [1]:
import tweepy
import json
from pymongo import MongoClient
from collections import Counter
from collections.abc import MutableMapping
import logging
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import community
import seaborn as sns
from fastprogress import master_bar, progress_bar
import time
import re


#Logger
logging.basicConfig(filename='PCA.log', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logging.getLogger().addHandler(ch)

In [ ]:
################
# RESTART DATA #
################

H = nx.read_gml("Andalusia-CI-Complete.gml")

print("Nombre de nodes: {}".format(H.number_of_nodes()))
print("Nombre d'arestes: {}".format(H.number_of_edges()))

with open('And-community_dict.json', 'r') as fp:
    community_dict = json.load(fp)
    
len(community_dict)

In [ ]:
################
# PREPARE DATA #
################


COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSOE", "POR_AND", "ADELANTE_AND"]
PARTY_NAMES_LIST = ['vox_es', 'populares','CiudadanosCs', 'PSOE', 'PODEMOS', 'AdelanteAND']


def load_ideologial_positions(file_name):
    with open(file_name, 'r') as fp:
        positions = json.load(fp)       
    return positions

positions = load_ideologial_positions('And-ideological_position_dict.json')
print(positions)
'''
matrix = np.zeros((len(COMMUNITIES_LIST),len(COMMUNITIES_LIST)))

for A in COMMUNITIES_LIST:
    for B in COMMUNITIES_LIST:
        a = np.array(positions[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(A)]])
        b = np.array(positions[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(B)]])
        distance = np.linalg.norm(a-b)
        #https://en.wikipedia.org/wiki/Euclidean_distance
                
        matrix[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = distance
        
print(matrix)
        '''

In [ ]:
#################################
# VISUALIZE (WIH PARTY NAMES)   #
#################################

COMMUNITIES_LIST = ["VOX", "POR_AND", "PSOE", "PP", "ADELANTE_AND", "CS"]
COLORS_LIST = ["darkgreen", "purple", "red", "blue", "lightgreen", "darkorange"]

def community_color(node_name):
    community = community_dict[node_name]
    index = COMMUNITIES_LIST.index(community)
    color = COLORS_LIST[index]
    return color

# Visualize embeddings in 2D space with node labels
plt.figure(figsize=(12, 10))
#plt.scatter(embedding[:, 0], embedding[:, 1], s=1)
#plt.scatter(positions.values(), c='blue', alpha=0.7)


'''
#Only label top10 in-degree nodes
indeg = G.in_degree(weight='weight')
sorted_indeg = sorted(indeg, key=lambda i: i[1], reverse=True)[:10]
print("Usuaris amb més grau d'ENTRADA:")
print(sorted_indeg)
sorted_indeg_names = [i[0] for i in sorted_indeg]
'''

ideological_position_dict ={}

# Add node labels
party_names = ['vox_es', 'PODEMOS', 'PSOE', 'populares', 'AdelanteAND', 'CiudadanosCs']
for node_name, position in positions.items():
    if position[0]<5000 and position [0]>-5000:
        if position[1]<5000 and position [1]>-5000:
            #plt.scatter(position[0], position[1], c='blue', s=1)
            plt.scatter(position[0], position[1], c=community_color(node_name), s=1)
            if node_name in party_names:
                print(node_name)
                print(position)
                ideological_position_dict [node_name] = [position[0], position[1]]
                #plt.plot(position[0], position[1], c='green', marker="X")
                #plt.text(position[0], position[1], node_name, fontsize=8, c='red')
                if node_name == 'PSOE':
                    plt.text(position[0], position[1], node_name, fontsize=8, c='blue')
                else:
                    plt.text(position[0], position[1], node_name, fontsize=8, c='red')
                

#plt.xlabel('X Dimension')
#plt.xlabel('Y Dimension')
plt.title('Andalusia - Visualization of RT Network')

plt.show()

In [ ]:
import pandas as pd

df_raw = pd.DataFrame(positions.items(), columns = [
                 "Party",
                 "Coordinates"
             ]
         )

df_raw

In [ ]:
df_raw[['X','Y']] = pd.DataFrame(df_raw.Coordinates.tolist(), index= df_raw.index)
df = pd.DataFrame(df_raw['Coordinates'].to_list(), columns=['X','Y'])
print (df)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
result = pca.fit_transform(df)
print(result)

In [ ]:
scaled_result = [x+5 for x in result]
print(scaled_result)